In [1]:
from typing import Optional


class BpmnProcessTransformer:
    """
    Class to transform the BPMN process structure generated by the LLM into a new structure more suitable for BPMN XML generation.
    """

    def transform(
        self, process: list[dict], parent_next_element_id: Optional[str] = None
    ) -> dict:
        """
        Restructure the original BPMN JSON structure into a new structure more suitable for BPMN XML generation.

        Example output structure::

            {
                "elements": [
                    {
                        "id": "element_id",
                        "type": "element_type",
                        "label": "element_label",
                        "incoming": ["incoming_flow_id"],
                        "outgoing": ["outgoing_flow_id"]
                    }
                ],
                "flows": [
                    {
                        "id": "flow_id",
                        "sourceRef": "source_element_id",
                        "targetRef": "target_element_id",
                        "condition": "flow_condition"
                    }
                ]
            }
        """

        elements: list[dict] = []
        flows: list[dict] = []

        def add_flow(source_ref, target_ref, flow_id=None, condition=None):
            """
            Helper function to append a flow to the flows list.
            """
            for flow in flows:
                if flow["sourceRef"] == source_ref and flow["targetRef"] == target_ref:
                    return

            flow_id = flow_id or f"{source_ref}-{target_ref}"
            flows.append(
                {
                    "id": flow_id,
                    "sourceRef": source_ref,
                    "targetRef": target_ref,
                    "condition": condition,
                }
            )

        def handle_exclusive_gateway(
            element: dict, next_element_id: Optional[str] = None
        ) -> Optional[str]:
            # If the exclusive gateway has a 'join' gateway, add it to the elements list
            join_gateway_id = None
            if element.get("has_join", False):
                join_gateway_id = f"{element['id']}-join"
                elements.append(
                    {
                        "id": join_gateway_id,
                        "type": "exclusiveGateway",
                        "label": None,
                    }
                )

            for branch in element["branches"]:

                if not branch.get("path"):
                    # Connect the exclusive gateway either to the next element in the process
                    # or to the branch's "next" element if specified
                    target_ref = branch.get("next", next_element_id)
                    if target_ref:
                        add_flow(
                            element["id"],
                            target_ref,
                            condition=branch.get("condition", None),
                        )
                    continue  # Skip further processing for empty branches

                branch_next = branch.get("next")

                if branch_next:
                    branch_structure = self.transform(branch["path"], branch_next)
                else:
                    branch_structure = self.transform(
                        branch["path"], join_gateway_id or next_element_id
                    )

                elements.extend(branch_structure["elements"])
                flows.extend(branch_structure["flows"])

                # Add the flow from the exclusive gateway to the first element in the branch
                first_element = (
                    branch_structure["elements"][0]
                    if branch_structure["elements"]
                    else None
                )
                if first_element:
                    add_flow(
                        element["id"],
                        first_element["id"],
                        condition=branch["condition"],
                    )

            return join_gateway_id

        def handle_parallel_gateway(element: dict) -> str:
            # Create a 'join' parallel gateway element
            join_gateway_id = f"{element['id']}-join"
            elements.append(
                {
                    "id": join_gateway_id,
                    "type": "parallelGateway",
                    "label": None,
                }
            )

            for branch in element["branches"]:
                branch_structure = self.transform(branch, join_gateway_id)
                elements.extend(branch_structure["elements"])
                flows.extend(branch_structure["flows"])

                # Add the flow from the parallel gateway to the first element in the branch
                first_element = branch_structure["elements"][0]
                add_flow(element["id"], first_element["id"])

                # Add the flow from the last element in the branch to the join gateway
                last_element = branch_structure["elements"][-1]
                add_flow(last_element["id"], join_gateway_id)

            return join_gateway_id

        for index, element in enumerate(process):
            next_element_id = (
                process[index + 1]["id"]
                if index < len(process) - 1
                else parent_next_element_id
            )

            elements.append(
                {
                    "id": element["id"],
                    "type": element["type"],
                    "label": element.get("label", None),
                }
            )

            if element["type"] == "exclusiveGateway":
                join_gateway_id = handle_exclusive_gateway(element, next_element_id)

                # Connect the join gateway to the next element in the process
                if join_gateway_id and next_element_id:
                    add_flow(join_gateway_id, next_element_id)
            elif element["type"] == "parallelGateway":
                join_gateway_id = handle_parallel_gateway(element)

                # Connect the join gateway to the next element in the process
                if next_element_id:
                    add_flow(join_gateway_id, next_element_id)
            elif next_element_id and element["type"] != "endEvent":
                # Add the flow between the current element and the next element in the process
                add_flow(element["id"], next_element_id)

        # Add incoming and outgoing flows to each element
        for element in elements:
            element["incoming"] = [
                flow["id"] for flow in flows if flow["targetRef"] == element["id"]
            ]
            element["outgoing"] = [
                flow["id"] for flow in flows if flow["sourceRef"] == element["id"]
            ]

        return {"elements": elements, "flows": flows}

In [2]:
import json
from typing import List

def load_process_from_file(path: str):
    """
    Load a BPMN process definition from a .json or .txt file.
    """
    with open(path, "r", encoding="utf-8") as f:
        content = f.read()

    # JSON Format
    if content.strip().startswith("{") or content.strip().startswith("["):
        process = json.loads(content)
    else:
        # evaluate Python-like text format
        process = eval(content)

    # If process is wrapped like {"process": [...]}
    if isinstance(process, dict) and "process" in process:
        return process["process"]
    
    return process

In [3]:
# 1) Prozess laden (z.B. deine Valid-example.txt)
process = load_process_from_file("Test2.txt")
print("Original process:")
print(json.dumps(process, indent=2))

# 2) Optional: nochmal validieren, um sicherzugehen
#validate_bpmn(process)
#print("\n✅ Prozess hat die Validierung bestanden.")

# 3) Transformer instanziieren und Prozess transformieren
transformer = BpmnProcessTransformer()
transformed = transformer.transform(process)

# 4) Ergebnis anzeigen
print("\nTransformed process structure:")
print(json.dumps(transformed, indent=2))


Original process:
[
  {
    "type": "startEvent",
    "id": "start",
    "label": "New pizza order received"
  },
  {
    "type": "task",
    "id": "task1",
    "label": "Make pizza"
  },
  {
    "type": "task",
    "id": "task2",
    "label": "Assess delivery route"
  },
  {
    "type": "exclusiveGateway",
    "id": "gateway1",
    "label": "Route less than one kilometer?",
    "has_join": false,
    "branches": [
      {
        "condition": "Route is less than one kilometer",
        "path": [
          {
            "type": "task",
            "id": "task3",
            "label": "Deliver pizza by foot"
          },
          {
            "type": "endEvent",
            "id": "end1",
            "label": "Pizza delivery completed"
          }
        ]
      },
      {
        "condition": "Route is one kilometer or more",
        "path": [
          {
            "type": "task",
            "id": "task4",
            "label": "Deliver pizza by bike"
          },
          {
      

In [4]:
import json
import xml.etree.ElementTree as ET

#neu
import requests
import re

BPMN_NS = "http://www.omg.org/spec/BPMN/20100524/MODEL"

def to_bpmn_prefixed(xml: str) -> str:
    """
    Converts default-namespace BPMN XML to explicit bpmn:-prefixed BPMN XML.
    This makes auto-layout generation of BPMNEdge/waypoints significantly more reliable.
    """
    if "xmlns:bpmn=" in xml:
        return xml  # already prefixed

    # Root tag: <definitions> -> <bpmn:definitions ... xmlns:bpmn="...">
    xml = re.sub(r"<definitions\b", f'<bpmn:definitions xmlns:bpmn="{BPMN_NS}"', xml, count=1)
    xml = re.sub(r"</definitions\s*>", "</bpmn:definitions>", xml, count=1)

    # Remove default BPMN xmlns if it matches BPMN_NS
    xml = re.sub(rf'\sxmlns="{re.escape(BPMN_NS)}"', "", xml, count=1)

    # Prefix common BPMN tags produced by your generator
    tags = [
        "process", "startEvent", "endEvent",
        "task", "userTask", "serviceTask",
        "exclusiveGateway", "parallelGateway", "inclusiveGateway",
        "eventBasedGateway", "subProcess", "callActivity",
        "sequenceFlow", "incoming", "outgoing"
    ]
    for t in tags:
        xml = re.sub(rf"<{t}\b", f"<bpmn:{t}", xml)
        xml = re.sub(rf"</{t}\b", f"</bpmn:{t}", xml)

    return xml
#endneu 

#from bpmn_assistant.config import logger
#from bpmn_assistant.services import BpmnProcessTransformer

import logging
logger = logging.getLogger("bpmn_xml")
logger.setLevel(logging.DEBUG)
if not logger.handlers:
    handler = logging.StreamHandler()
    handler.setLevel(logging.DEBUG)
    logger.addHandler(handler)

class BpmnXmlGenerator:
    """
    Class to generate BPMN XML from the BPMN process data in JSON format.
    """

    def __init__(self, layout_server_url: str = "http://localhost:3001/process-bpmn", timeout_s: int = 60):
        self.transformer = BpmnProcessTransformer()
        self.layout_server_url = layout_server_url
        self.timeout_s = timeout_s


    def create_bpmn_xml(self, process: list[dict]) -> str:
        """
        Create BPMN XML from the process data.
        Args:
            process: BPMN process structure generated by the LLM.
            The BPMN XML string.
        """

        transformed_process = self.transformer.transform(process)
        logger.debug(
            f"Transformed process:\n{json.dumps(transformed_process, indent=2)}"
        )

        # Create the root element (definitions)
        root = ET.Element("definitions")
        root.set("xmlns", "http://www.omg.org/spec/BPMN/20100524/MODEL")
        root.set("xmlns:bpmndi", "http://www.omg.org/spec/BPMN/20100524/DI")
        root.set("xmlns:dc", "http://www.omg.org/spec/DD/20100524/DC")
        root.set("xmlns:di", "http://www.omg.org/spec/DD/20100524/DI")
        root.set("id", "definitions_1")
        root.set("targetNamespace", "http://bpmn.io/schema/bpmn")  #neu


        # Create the process element
        process_element = ET.SubElement(root, "process")
        process_element.set("id", "Process_1")
        process_element.set("isExecutable", "false")

        # Add elements
        for element in transformed_process["elements"]:
            elem = ET.SubElement(process_element, element["type"])
            elem.set("id", element["id"])

            # Add label if it exists
            if element["label"]:
                elem.set("name", element["label"])

            # Add incoming and outgoing flows as child elements
            for incoming in element["incoming"]:
                ET.SubElement(elem, "incoming").text = incoming
            for outgoing in element["outgoing"]:
                ET.SubElement(elem, "outgoing").text = outgoing

        # Add flows
        for flow in transformed_process["flows"]:
            seq_flow = ET.SubElement(process_element, "sequenceFlow")
            seq_flow.set("id", flow["id"])
            seq_flow.set("sourceRef", flow["sourceRef"])
            seq_flow.set("targetRef", flow["targetRef"])

            # Add condition if it exists
            if flow["condition"]:
                seq_flow.set("name", flow["condition"])

        xml_string = ET.tostring(root, encoding="unicode") #neu

        # 1) Preprocessing: enforce bpmn: prefixes to make auto-layout edges reliable
        xml_for_layout = to_bpmn_prefixed(xml_string)

        # 2) Call layout server to add BPMNDI (Shapes + Edges + Waypoints)
        resp = requests.post(
            self.layout_server_url,
            json={"bpmnXml": xml_for_layout},
            timeout=self.timeout_s
        )
        resp.raise_for_status() 
        data = resp.json()

        # Expected: {"layoutedXml": "..."}
        layouted_xml = data.get("layoutedXml")
        if not layouted_xml:
            # fail fast with a helpful message
            raise ValueError(f"Layout server response missing 'layoutedXml'. Keys: {list(data.keys())}")    
        
        return layouted_xml #endneu

In [5]:
import json  # falls noch nicht importiert

# 1) Prozess laden (z.B. deinen validen Beispielprozess)
process = load_process_from_file("Test2.txt")
print("Original process:")
print(json.dumps(process, indent=2))

# 2) Optional: nochmal validieren, um sicherzugehen
#validate_bpmn(process)
#print("\n✅ Prozess hat die Validierung bestanden.")

# 3) XML-Generator instanziieren
xml_generator = BpmnXmlGenerator()

# 4) BPMN-XML erzeugen
layouted_xml = xml_generator.create_bpmn_xml(process)

# 5) XML anzeigen
print("\nGenerated BPMN XML:")
print(layouted_xml)

# 6) (optional) in Datei speichern, damit du es später im Modeler öffnen kannst
with open("generated_process_new.bpmn", "w", encoding="utf-8") as f:
    f.write(layouted_xml)

print("\n💾 BPMN XML wurde in 'generated_process_new.bpmn' gespeichert.")

Transformed process:
{
  "elements": [
    {
      "id": "start",
      "type": "startEvent",
      "label": "New pizza order received",
      "incoming": [],
      "outgoing": [
        "start-task1"
      ]
    },
    {
      "id": "task1",
      "type": "task",
      "label": "Make pizza",
      "incoming": [
        "start-task1"
      ],
      "outgoing": [
        "task1-task2"
      ]
    },
    {
      "id": "task2",
      "type": "task",
      "label": "Assess delivery route",
      "incoming": [
        "task1-task2"
      ],
      "outgoing": [
        "task2-gateway1"
      ]
    },
    {
      "id": "gateway1",
      "type": "exclusiveGateway",
      "label": "Route less than one kilometer?",
      "incoming": [
        "task2-gateway1"
      ],
      "outgoing": [
        "gateway1-task3",
        "gateway1-task4"
      ]
    },
    {
      "id": "task3",
      "type": "task",
      "label": "Deliver pizza by foot",
      "incoming": [
        "gateway1-task3"
      ],
  

Original process:
[
  {
    "type": "startEvent",
    "id": "start",
    "label": "New pizza order received"
  },
  {
    "type": "task",
    "id": "task1",
    "label": "Make pizza"
  },
  {
    "type": "task",
    "id": "task2",
    "label": "Assess delivery route"
  },
  {
    "type": "exclusiveGateway",
    "id": "gateway1",
    "label": "Route less than one kilometer?",
    "has_join": false,
    "branches": [
      {
        "condition": "Route is less than one kilometer",
        "path": [
          {
            "type": "task",
            "id": "task3",
            "label": "Deliver pizza by foot"
          },
          {
            "type": "endEvent",
            "id": "end1",
            "label": "Pizza delivery completed"
          }
        ]
      },
      {
        "condition": "Route is one kilometer or more",
        "path": [
          {
            "type": "task",
            "id": "task4",
            "label": "Deliver pizza by bike"
          },
          {
      